In [1]:
%%writefile conf/config.yaml
# Arch	                        Start Lr	Max Steps	Decay Steps
# FullyConnectedArch	        1.00E-03	1500000	        15000	   
defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

scheduler:
  decay_rate: 0.95
  decay_steps: 100

training:
  rec_results_freq: 1000
  max_steps : 10000

batch_size:
  IC: 315
  BC: 200
  interior: 3150

Overwriting conf/config.yaml


In [2]:
%%writefile fhnx1d.py
import numpy as np
from sympy import Symbol, sin

import modulus
from modulus.hydra import instantiate_arch, ModulusConfig
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Line1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.eq.pde import PDE
from modulus.models.fully_connected import FullyConnectedArch

from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
from sympy import Symbol, Function, Number
from modulus.eq.pde import PDE

import numpy as np
from sympy import Symbol, sin

import modulus
from modulus.hydra import instantiate_arch, ModulusConfig
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Line1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.utils.io.plotter import ValidatorPlotter

from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
import matplotlib.pyplot as plt
from modulus.domain.validator.discrete import GridValidator
from modulus.dataset.discrete import DictGridDataset
from modulus.domain.constraint.continuous import DeepONetConstraint

class plotter(ValidatorPlotter):
    "Default plotter class for validator"

    def __call__(self, invar, true_outvar, pred_outvar):
        "Default function for plotting validator data"

        # interpolate 2D data onto grid
        print(len(invar))
        print(np.shape((pred_outvar["u"].flatten())))
        print(len(pred_outvar))
        
        
        invar["K"]=invar["K"]*50
        #print(invar["K"])
        true_outvar["u"]=np.expand_dims(true_outvar["u"], axis=1) 
        
        extent, true_outvar, pred_outvar = self._interpolate_2D(
                200, invar, true_outvar, pred_outvar
            )
        ndim=2
        # make plots
        dims = list(invar.keys())
        fs = []
        print("kk")
        for k in pred_outvar:
            f = plt.figure(figsize=(3 * 5, 4), dpi=100)
            for i, (o, tag) in enumerate(
                zip(
                    [true_outvar[k], pred_outvar[k], ((true_outvar[k] - pred_outvar[k])**2)**(0.5) ],
                    ["true", "pred", "diff"],
                )
            ):
           #     print("extent",extent)
                plt.subplot(1, 3, 1 + i)
                if ndim == 1:
                    plt.plot(invar[dims[0]][:, 0], o[:, 0])
                    plt.xlabel(dims[0])
                elif ndim == 2:
                    plt.imshow(o.T, origin="lower", extent=extent)
                    
                    plt.xlabel(dims[0])
                    plt.ylabel(dims[1])
                    if(tag=="diff"):
                       
                        plt.clim(0,0.3)
                    plt.colorbar()
                plt.title(f"{k}_{tag}")
            plt.tight_layout()
            fs.append((f, k))

        return fs




def generateExactSolution(t,dt,x0,rate,P):
    
    n2=int(t/dt)
    n = int(t/(dt*rate))
    Sol=np.zeros(n)
    Sol[0]=x0
    Sol2=np.zeros(n2)
    Sol2[0]=x0
    T=0
    k=0
    while(k<n2-1):
        x=Sol2[k]
        Sol2[k+1]=x*(x-0.2)*(1-x)*dt +  x
        if ((k+1)%rate == 0):
            T=T+1
            Sol[T] = Sol2[k+1]         
        k=k+1
    return Sol
class WaveEquation1D(PDE):
   

    name = "WaveEquation1D"

    def __init__(self, c=1.0):
        
        
        t = Symbol("x")
        input_variables = {"x": t}

        x1 = Function("u")(*input_variables)

        self.equations = {}
        self.equations["ode"] = x1*(1-x1)*(x1-0.2) -x1.diff(t)
        

        



u0=0.6

@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:

    # make list of nodes to unroll graph on
    we = WaveEquation1D(c=1.0)
    wave_net =FullyConnectedArch(
            input_keys=[Key("x"), Key("K")],
            output_keys=[Key("u")],
        ) 

    nodes = we.make_nodes() + [wave_net.make_node(name="wave_network")]

    # add constraints to solver
    # make geometry
    x, k_symbol = Symbol("x"), Symbol("K")
    t_max=L = 10.0
    geo = Line1D(0, L)
    k_range = {k_symbol: (0.3, 0.5)}

    # make domain
    domain = Domain()

    # initial condition
   # IC = PointwiseInteriorConstraint(
   #     nodes=nodes,
    #    geometry=geo,
   #     outvar={"u": sin(x), "u__t": sin(x)},
    #    batch_size=cfg.batch_size.IC,
    #    lambda_weighting={"u": 1.0, "u__t": 1.0},
     #   parameterization={x: (0.0,0.2)},
    #)
   # domain.add_constraint(IC, "IC")

    # boundary condition
    
    #BC = PointwiseBoundaryConstraint(
    #    nodes=nodes,
    #    geometry=geo,
    #    outvar={"u": 0},
    #    batch_size=cfg.batch_size.BC,
    #    parameterization=time_range,
    #)
    #domain.add_constraint(BC, "BC")

    # interior
    interior = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode": 0},
        batch_size=cfg.batch_size.interior,
        parameterization=k_range,
    )
    domain.add_constraint(interior, "interior")

    
    ##DD constraint
    
    T=np.empty([0])
    K=np.empty([0])
    SOLs=np.empty([0])
    krange= [(0.3 + 0.01*i*0.5) for i in range(1,10)]

    deltaT = 0.01
    rate = 100
    t = np.linspace(0, t_max, int((t_max/(deltaT)) /rate) )
    t = np.expand_dims(t, axis=-1)
    
    for KR in krange:
        
        
        T=np.append(T,t)
        K = np.append(K,np.full_like (t,KR))
        SOLs=np.append(SOLs,np.array(generateExactSolution(t_max,deltaT,KR,rate,KR)))
    
    
    
    t = np.expand_dims(T, axis=-1)
  


    k = np.expand_dims(K, axis=-1)

    
    Sol = np.expand_dims(SOLs, axis=-1)

    print(np.shape(t),"training set")
    
    
    invar_numpy = {"t": t,"k":k}
    outvar_numpy = {
        "x1": Sol
    }

    
    data = DeepONetConstraint.from_numpy(
        nodes=nodes,
        invar={"x":t,"K":k},
        outvar={"u":Sol},
        batch_size=20,
    )
    domain.add_constraint(data, "data")
    
    
    
    
    ##validator
    deltaT = 0.01
    rate = 10
    t = np.linspace(0, t_max, int((t_max/(deltaT)) /rate) )
    t = np.expand_dims(t, axis=-1)
    for KR in krange:
        
        
        T=np.append(T,t)
        K = np.append(K,np.full_like (t,KR))
        SOLs=np.append(SOLs,np.array(generateExactSolution(t_max,deltaT,KR,rate,0)))
    
    
    t=np.expand_dims(T,axis=-1)
    k=np.expand_dims(K,axis=-1)
    sol=SOLs
    
    invar_numpy = {"x": t,"K":k}
    outvar_numpy = {
        "u": sol
    }
    
  
    
    validator = PointwiseValidator(
        nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy, batch_size=1024,plotter= plotter()
    )
    domain.add_validator(validator)
    
    
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()



if __name__ == "__main__":
    run()


Overwriting fhnx1d.py


In [3]:
!rm -r outputs/fhnx1d || true ##se não limpar o output ele aproveita o treinamento, mesmo se mudar o modelo
!python fhnx1d.py


[22:46:53] - JIT using the NVFuser TorchScript backend
[22:46:53] - JitManager: {'_enabled': True, '_arch_mode': <JitArchMode.ONLY_ACTIVATION: 1>, '_use_nvfuser': True, '_autograd_nodes': False}
[22:46:53] - GraphManager: {'_func_arch': False, '_debug': False, '_func_arch_allow_partial_hessian': True}
(90, 1) training set
[22:46:54] - Installed PyTorch version 1.13.1+cu117 is not TorchScript supported in Modulus. Version 1.13.0a0+d321be6 is officially supported.
[22:46:55] - attempting to restore from: outputs/fhnx1d
[22:46:55] - optimizer checkpoint not found
[22:46:55] - model wave_network.0.pth not found
[22:46:56] - [step:          0] record constraint batch time:  4.630e-01s
2
(990,)
1
kk
[22:46:57] - [step:          0] record validators time:  1.009e+00s
[22:46:57] - [step:          0] saved checkpoint to outputs/fhnx1d
[22:46:57] - [step:          0] loss:  8.452e+00
[22:46:59] - Attempting cuda graph building, this may take a bit...
[22:47:01] - [step:        100] loss:  4.337e

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
def find(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

base_dir = "outputs/fhnx1d/validators/"

data = np.loadtxt(base_dir + "validator.vtp")
data = np.atleast_1d(data.f.arr_0)[0]

def interpolate_2D( size, invar, *outvars):
        "Interpolate 2D outvar solutions onto a regular mesh"

        print(len(invar))
        assert len(invar) == 2

        # define regular mesh to interpolate onto
        xs = [invar[k][:, 0] for k in invar]
        extent = (xs[0].min(), xs[0].max(), xs[1].min(), xs[1].max())
        xyi = np.meshgrid(
            np.linspace(extent[0], extent[1], size),
            np.linspace(extent[2], extent[3], size),
            indexing="ij",
        )

        # interpolate outvars onto mesh
        outvars_interp = []
        for outvar in outvars:
            outvar_interp = {}
            for k in outvar:
                outvar_interp[k] = scipy.interpolate.griddata(
                    (xs[0], xs[1]), outvar[k][:, 0], tuple(xyi)
                )
            outvars_interp.append(outvar_interp)

        return [extent] + outvars_interp


def call(invar, true_outvar, pred_outvar):



        # interpolate 2D data onto grid
        print(len(invar))
        extent, true_outvar, pred_outvar = interpolate_2D(
                200, invar, true_outvar, pred_outvar
            )
        ndim=2
        # make plots
        dims = list(invar.keys())
        fs = []
        for k in pred_outvar:
            f = plt.figure(figsize=(3 * 5, 4), dpi=100)
            for i, (o, tag) in enumerate(
                zip(
                    [true_outvar[k], pred_outvar[k], ((true_outvar[k] - pred_outvar[k])**2)**(0.5) ],
                    ["true", "pred", "diff"],
                )
            ):
                plt.subplot(1, 3, 1 + i)
                if ndim == 1:
                    plt.plot(invar[dims[0]][:, 0], o[:, 0])
                    plt.xlabel(dims[0])
                elif ndim == 2:
                    plt.imshow(o.T, origin="lower", extent=extent)
                    
                    plt.xlabel(dims[0])
                    plt.ylabel(dims[1])
                    if(tag=="diff"):
                       
                        plt.clim(0,0.3)
                    plt.colorbar()
                plt.title(f"{k}_{tag}")
            plt.tight_layout()
            fs.append((f, k))
     
    
    
invar={
       "t":data["t"],"K":data["K"]*15
      }
print(len(invar))
out={"x1":data["pred_x1"],
     }
out_t={"x1":data["true_x1"],
      }
call(invar,out_t,out)


ValueError: could not convert string to float: '<?xml'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def find(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

base_dir = "outputs/fhnx0dd/validators/"

# plot in 1d
data = np.load(base_dir + "validator.npz", allow_pickle=True)
data = np.atleast_1d(data.f.arr_0)[0]

ks=np.unique(data["K"])
t=np.unique(data["t"])
d=np.full_like(ks,0)

for k in ks:
    i=find(data["K"],k)
    #print(i)
    x=data["true_x1"][i]
    pred=data["pred_x1"][i]
    d=np.mean(x-pred)
    plt.ylim(0,1.1)
    plt.plot(t,x,"o",label="True ,k ="+ str(k))
    plt.plot(t,pred,label="Pred ,k ="+ str(k))
    plt.legend(loc="best")
    plt.show()
    